In [1]:
% pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [67]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

import itertools

# %matplotlib inline 

In [70]:
df = pd.read_csv('planets.csv')

In [101]:
exodf = df.loc[(df['st_mass']<= 0.6) & (df['st_teff'] < 6000) & (df['pl_orbsmax'] > 0)]
stardf = df.loc[(df['st_mass']<= 0.6) & (df['st_teff'] < 6000) & (df.duplicated('pl_hostname')) & (df['pl_orbsmax'] > 0)]

In [72]:
bitwise_mask = (np.isfinite(stardf['st_dist'])) & (np.isfinite(starexodf['st_mass'])) & (np.isfinite(stardf['st_teff']))

In [73]:
ra = stardf['ra'].loc[bitwise_mask]
dec = stardf['dec'].loc[bitwise_mask]
distance = stardf['st_dist'].loc[bitwise_mask] 
stellar_mass = stardf['st_mass'].loc[bitwise_mask]
teff = stardf['st_teff'].loc[bitwise_mask]
star_name = stardf['pl_hostname'].loc[bitwise_mask]

In [74]:
x = distance * np.cos(dec) * np.cos(ra)
y = distance * np.cos(dec) * np.sin(ra)
z = distance * np.sin(dec)

In [29]:
#axes limits for on_key codes

a = -300
b = 300
c = a
d = b
e = a
f = b

In [ ]:
labels_column = star_name
labels = labels_column.values

In [ ]:
colors = itertools.cycle(['#42f4bf', 'green', 'blue']) #these should come from Luna's colors

In [59]:
fig = plt.figure(figsize=[10, 8]) 

ax = fig.add_subplot(111, facecolor='Black', projection='3d', xlim = (a,b), ylim = (c,d), zlim = (e,f))
ax.scatter(x, y, z, c = teff, cmap= 'plasma', s = 2*((10*stellar_mass)**2), marker = 'o')  
ax.set_axis_off()

fig = zoomandpan_factory(ax,base_scale = scale) #instead of this, we should connect the plots to everyone's code with mpl connect

In [58]:
def zoomandpan_factory(ax,base_scale = 2.):
    
    #panning
    def panning_fun(event): #creates our event handling function to move left on the x-axis by subtracting 5
        if event.key == 'left': #if the key pressed is the left arrow
            global a #tells our function that we defined 'a' outside of the function
            global b #^
            a = a-50 #subtracts 50 from the value of a each time the key is pressed
            b = b-50 #^
            ax.set_xlim([a,b]) #resets the x-axis limits to new 'a' and 'b'
            plt.show() #re-shows the plot so we can see the change in axis

        if event.key == 'right': #if the key pressed is the right arrow
            a = a+50 #adds 50 from the value of a each time the key is pressed
            b = b+50 #^
            ax.set_xlim([a,b]) #resets the x-axis limits to new 'a' and 'b'
            plt.show() #re-shows the plot so we can see the change in axis

        if event.key == 'down': #if the key pressed is the down arrow
            global c #tells our function that we defined 'c' outside of the function
            global d #^
            c = c-50 #subtracts 50 from the value of a each time the key is pressed
            d = d-50 #^
            ax.set_ylim([c,d]) #resets the y-axis limits to new 'c' and 'd'
            plt.show() #re-shows the plot so we can see the change in axis

        if event.key == 'up': #if the key pressed is the up arrow
            c = c+50 #adds 50 from the value of a each time the key is pressed
            d = d+50 #^
            ax.set_ylim([c,d]) #resets the y-axis limits to new 'c' and 'd'
            plt.show() #re-shows the plot so we can see the change in axis

        if event.key == 'r': #if the key pressed is the r key
            global e #tells our function that we defined 'e' outside of the function
            global f #^
            e = e-50 #subtracts 50 from the value of a each time the key is pressed
            f = f-50 #^
            ax.set_zlim([e,f]) #resets the y-axis limits to new 'c' and 'd'
            plt.show() #re-shows the plot so we can see the change in axis

        if event.key == 't': #if the key pressed is the t key
            e = e+50 #adds 50 from the value of a each time the key is pressed
            f = f+50 #^
            ax.set_zlim([e,f]) #resets the y-axis limits to new 'c' and 'd'
            plt.show() #re-shows the plot so we can see the change in axis

    fig = ax.get_figure()
    fig.canvas.mpl_connect('key_press_event', panning_fun) 
    #connects our figure with our above function; tells the figure that it should expect a key_press

    
    def zoom_fun(event):
        base_scale =1.5
        
        # get the current x and y limits
        cur_xlim = ax.get_xlim()
        cur_ylim = ax.get_ylim()
        cur_xrange = (cur_xlim[1] - cur_xlim[0])*.5
        cur_yrange = (cur_ylim[1] - cur_ylim[0])*.5
        xdata = event.xdata # get event x location
        ydata = event.ydata # get event y location
        if event.button == 'up':
            # deal with zoom in
            scale_factor = 1/base_scale
        elif event.button == 'down':
            # deal with zoom out
            scale_factor = base_scale
        else:
            # deal with something that should never happen
            scale_factor = 1
            print (event.button)
        # set new limits
        ax.set_xlim([xdata - cur_xrange*scale_factor,
                     xdata + cur_xrange*scale_factor])
        ax.set_ylim([ydata - cur_yrange*scale_factor,
                     ydata + cur_yrange*scale_factor])
        plt.draw() # force re-draw

    fig = ax.get_figure() # get the figure of interest
    # attach the call back
    fig.canvas.mpl_connect('scroll_event',zoom_fun)
